### Import Numpy

In [1]:
import numpy as np

### Define Tree Object

In [37]:
class CartTree:
    
    def fit(self, X, y, min_leaf = 5):
        self.dtree = Node(X, y, np.array(np.arange(len(y))), min_leaf)
        return self
        
    def predict(self, X):
        return self.dtree.predict(X)

### Define Node Object

In [38]:
class Node:
    
    def __init__(self, x, y, idxs, min_leaf = 5):
        self.x = x
        self.y = y
        self.idxs = idxs
        self.min_leaf = min_leaf
        self.row_count = len(idxs)
        self.col_count = x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
        
    def find_varsplit(self):
        for c in range(self.col_count):
            self.find_better_split(c)
        if self.is_leaf:
            return
        x = self.split_col
        lhs = np.nonzero(x <= self.split)[0]
        rhs = np.nonzero(x > self.split)[0]
        self.lhs = Node(self.x, self.y, self.idxs[lhs], self.min_leaf)
        self.rhs = Node(self.x, self.y, self.idxs[rhs], self.min_leaf)
        
    @property
    def split_col(self):
        return self.x[self.idxs, self.var_idx]

    @property
    def is_leaf(self):
        return self.score == float('inf')
    
    def find_score(self, lhs, rhs):
        y = self.y[self.idxs]
        lhs = y[lhs].std()
        rhs = y[rhs].std()
        return lhs * lhs.sum() + rhs * rhs.sum()
    
    def find_better_split(self, var_idx):
        x = self.x[self.idxs, var_idx]

        for r in range(self.row_count):
            lhs = x <= x[r]
            rhs = x > x[r]
            if rhs.sum() < self.min_leaf or lhs.sum() < self.min_leaf:
                continue
            curr_score = self.find_score(lhs, rhs)
            if curr_score < self.score:
                self.var_idx = var_idx
                self.score = curr_score
                self.split = x[r]
                
    def predict_row(self, xi):
        if self.is_leaf:
            return self.val
        node = self.lhs if xi[self.var_idx] <= self.split else self.rhs
        return node.predict_row(xi)
    
    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

In [41]:
X = np.array([[2.771244718,1.784783929],
                [1.728571309,1.169761413],
                [3.678319846,2.81281357],
                [3.961043357,2.61995032],
                [2.999208922,2.209014212],
                [7.497545867,3.162953546],
                [9.00220326,3.339047188],
                [7.444542326,0.476683375],
                [10.12493903,3.234550982],
                [6.642287351,3.319983761]])

y = np.array([2, 3, 4, 2, 3, 5, 6, 7, 5, 4])

In [53]:
test = CartTree().fit(X, y, min_leaf = 2)
test.predict(X)

test.dtree.lhs.lhs.lhs.lhs

AttributeError: 'Node' object has no attribute 'lhs'